In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from agent.model import TD3_Actor
from environment.env import TradingEnv
from pykalman import KalmanFilter
import matplotlib.pyplot as plt

g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
env = TradingEnv('data')

g:\Code\Python\GitHub\Portfolio-Optimizer\environment\env.py:246: DeprecationWarning: This function is deprecated. Please call randint(59, 840 + 1) instead
  self.dates_num-self.steps-1)


In [41]:
env.observation_space.sample()

OrderedDict([('action',
              array([0.67052394, 0.27236295, 0.72192204, 0.10693281, 0.8211192 ,
                     0.57508737, 0.73986304, 0.9148778 ], dtype=float32)),
             ('portfolio',
              array([[[ 1.08218372e+00,  1.26657915e+00,  1.46539116e+00,
                       -7.02903926e-01,  1.29338741e-01, -7.21894920e-01,
                       -3.37756097e-01, -6.72622681e-01, -1.32207882e+00,
                       -9.96173024e-01, -3.70011538e-01,  2.95486242e-01,
                       -1.23627818e+00, -4.05236781e-01, -7.29962349e-01,
                       -8.02624285e-01, -3.61549020e-01,  6.94678485e-01,
                        5.61743200e-01,  1.51608959e-01,  1.33328700e+00,
                        3.45734984e-01, -3.53939950e-01,  9.77712214e-01,
                       -6.70196354e-01,  1.66448367e+00, -9.01000202e-02,
                       -8.43414843e-01, -9.05639350e-01,  1.80981767e+00,
                        1.62992405e-03,  9.96315107e-